In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('.')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/anc')

In [ ]:
!pip install import_ipynb

In [ ]:
name = "7th_ELU"

In [ ]:
import torchaudio
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import librosa
from torch.nn.utils.rnn import pad_sequence
from scipy.signal import fftconvolve
import torch.nn.functional as F

import matplotlib.pyplot as plt
import IPython.display as ipd
import import_ipynb

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
from model import CRN_2
from dataset import Dataset

In [ ]:
sampling_rate = 16000
length = sampling_rate*5

In [ ]:
ANC = CRN_2().to(device)

In [ ]:
from torch.utils.data import Subset
train_data = Dataset(train=True)
eval_data = Dataset(train=False)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
eval_dataloader = DataLoader(eval_data, batch_size=64, shuffle=False)

In [ ]:
learning_rate = 1e-3
epochs = 15
optimizer = torch.optim.AdamW(ANC.parameters(),lr = learning_rate)

In [ ]:
def rir_batch(wav: torch.Tensor, rir_path: str) -> torch.Tensor:
        rir_np = np.loadtxt(rir_path)[:, 1]
        rir_energy = np.sum(rir_np ** 2)
        rir_np = rir_np / np.sqrt(rir_energy)
        rir_tensor = torch.from_numpy(rir_np).float().to(wav.device)  # (R,)
        rir_tensor = rir_tensor.view(1, 1, -1)  # (1, 1, R) for conv1d
        padding = rir_tensor.size(-1) - 1
        # wav: (B, T) -> (B, 1, T)
        wav = wav.unsqueeze(1)
        # conv1d는 kernel을 뒤집으므로 flip
        rir_tensor_flipped = torch.flip(rir_tensor, dims=[2])
        # F.conv1d는 valid convolution 수행
        y = F.conv1d(wav, rir_tensor_flipped, padding=padding)  # (B, 1, T+R-1)
        y = y.squeeze(1)  # (B, T+R-1)

        return y[:, :length]

In [ ]:
import torch.nn.functional as F
epsilon = 1e-8
n_fft = 320
hop_length = 160
win_length = 320
window = torch.hann_window(win_length).to(device)  # STFT/ISTFT에 사용

train_loss = []
train_NMSE = []
eval_loss = []
eval_NMSE = []
epsilon = 1e-8

for epoch in range(epochs):
    print(f"Epoch {epoch}")

    train_loss_epoch = []
    train_nmse_epoch = []
    #--------------------------------------------------------------------------------------------------
    ANC.train()
    for ori, perr in train_dataloader:
        B = ori.size(0)
        ori = ori.to(device)
        perr = perr.to(device)

        optimizer.zero_grad()
        stft_ori = torch.stft(
            ori, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
            return_complex=True, center=True, onesided=True, window=window
        )
        stft_ori_real = stft_ori.real
        stft_ori_imag = stft_ori.imag
        stft_ori_tensor = torch.stack([stft_ori_real, stft_ori_imag], dim=1) # 원하는 사이즈 batch,2,time,320
        stft_pred = ANC(stft_ori_tensor)
        stft_pred_complex = torch.complex(stft_pred[:, 0], stft_pred[:, 1])  # real, imag
        # ISTFT
        istft_pred = torch.istft(
            stft_pred_complex,
            n_fft=n_fft,
            hop_length=hop_length,
            win_length=win_length,
            center=True,
            onesided=True,
            window=window,
            length=ori.shape[1]
        )

        u_serr = rir_batch(istft_pred,"s-err.txt")
        error  = perr + u_serr                         # 목표 0
        loss   = torch.mean(error ** 2)
        loss.backward()
        optimizer.step()
    #-------------------------------------------------------------------------------------------------------------
        train_loss_epoch.append(loss.item())
        # 배치별 NMSE 계산 후 저장
        for i in range(B):
            d = u_serr[i]       # 목표=0이므로 target이 곧 분모 신호
            err_i  = error[i]                  # = perr[i] + u_serr[i]

            num   = torch.sum(err_i ** 2)
            denom = torch.sum(d ** 2) + epsilon
            nmse_i = 10 * torch.log10(num / denom)
            train_nmse_epoch.append(nmse_i)

    # 에포크별 평균 기록
    epoch_loss = sum(train_loss_epoch) / len(train_loss_epoch)
    print(f"train_epoch_loss:{epoch_loss}")
    epoch_nmse = torch.mean(torch.stack(train_nmse_epoch)).item()
    print(f"train_epoch_NMSE:{epoch_nmse}")
    train_loss.append(epoch_loss)
    train_NMSE.append(epoch_nmse)
    #-------------------------------------------------------------------------------------------------------------
    eval_loss_epoch = []
    eval_nmse_epoch = []
    #-------------------------------------------------------------------------------------------------------------
    ANC.eval()

    for ori, perr in eval_dataloader:
        B = ori.size(0)                       # 배치 크기
        ori = ori.to(device)
        perr = perr.to(device)
        with torch.no_grad():
            stft_ori = torch.stft(
                ori, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
                return_complex=True, center=True, onesided=True, window=window
            )
            stft_ori_real = stft_ori.real
            stft_ori_imag = stft_ori.imag
            stft_x_tensor = torch.stack([stft_ori_real, stft_ori_imag], dim=1) # 원하는 사이즈 batch,2,time,320
            stft_pred = ANC(stft_x_tensor)
            stft_pred_complex = torch.complex(stft_pred[:, 0], stft_pred[:, 1])  # real, imag
            # ISTFT
            istft_pred = torch.istft(
                stft_pred_complex,
                n_fft=n_fft,
                hop_length=hop_length,
                win_length=win_length,
                center=True,
                onesided=True,
                window=window,
                length=ori.shape[1]
            )
            u_serr = rir_batch(istft_pred,"s-err.txt")
            error  = perr + u_serr                         # 목표 0
            loss   = torch.mean(error ** 2)
    #-------------------------------------------------------------------------------------------------------------
            eval_loss_epoch.append(loss.item())

        # 배치별 NMSE 계산 후 저장
        for i in range(B):
            d = u_serr[i]       # 목표=0이므로 target이 곧 분모 신호
            err_i  = error[i]                  # = perr[i] + u_serr[i]

            num   = torch.sum(err_i ** 2)
            denom = torch.sum(d ** 2) + epsilon
            nmse_i = 10 * torch.log10(num / denom)
            eval_nmse_epoch.append(nmse_i)

    # 에포크별 평균 기록
    epoch_loss = sum(eval_loss_epoch) / len(eval_loss_epoch)
    print(f"eval_epoch_loss:{epoch_loss}")
    epoch_nmse = torch.mean(torch.stack(eval_nmse_epoch)).item()
    print(f"eval_epoch_NMSE:{epoch_nmse}")
    eval_loss.append(epoch_loss)
    eval_NMSE.append(epoch_nmse)

In [ ]:
import matplotlib.pyplot as plt


epochs = list(range(1, len(train_loss) + 1))

# 그래프 출력
plt.figure(figsize=(12, 5))

# Loss 그래프
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label='Train Loss')
plt.plot(epochs, eval_loss, label='Eval Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Evaluation Loss')
plt.legend()
plt.grid(True)

# NMSE 그래프
plt.subplot(1, 2, 2)
plt.plot(epochs, train_NMSE, label='Train NMSE')
plt.plot(epochs, eval_NMSE, label='Eval NMSE')
plt.xlabel('Epoch')
plt.ylabel('NMSE (dB)')
plt.title('Training and Evaluation NMSE')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(f"{name}.png", dpi=300)
plt.show()

In [ ]:
torch.save(ANC.state_dict(), f'{name}.pth')